In [35]:
import urllib.parse
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, FOAF, OWL, XSD, DC, DCTERMS 
import pandas as pd

In [2]:
base_uri = "http://www.sanitasicilia.it/resource/"

g = Graph()

sso = Namespace("http://www.sanitasicilia.it/ontology/")

g.bind("sso", sso)

ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)


In [7]:
#Comuni
def urify(ns, testo):
    testo=testo.replace(" ","_").replace("\'","")
    return ns+urllib.parse.quote(testo)

def addTriples(row):
    res = URIRef(urify(base_uri, row[0]))
    g.add([res, URIRef(RDF.type), URIRef(sso.Comune)])
    g.add([res, sso.hasName, Literal(row[0], datatype=XSD.string)])
    g.add([res, sso.hasTotalPopulation, Literal(row[1], datatype=XSD.integer)])
    g.add([res, sso.hasProvince, Literal(row[2], datatype=XSD.string)])
    g.add([res, sso.hasProvinceAcr, Literal(row[3], datatype=XSD.string)])
    #interlinking
    

comuni_df = pd.read_csv("../datasets/completed/comuni_sicilia.csv")

comuni_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination='../datasets/rdf/comuni_sicilia.ttl', 
            format='turtle')

<Graph identifier=N2f592c4e4ccc49e780f2be41214f59e0 (<class 'rdflib.graph.Graph'>)>

In [38]:
#Farmacie
def addTriples(row):
    #sostituire con codice farmacia/parafarmacia piuttosto che nome 
    res = URIRef(urify(base_uri, row[5]))
    g.add([res, URIRef(RDF.type), URIRef(sso.Farmacia)])
    g.add([res, sso.isIn, URIRef(urify(base_uri, row[2].title()))])

    if(row[3] != 0):
        g.add([res, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
        
    g.add([res, sso.hasAddress, Literal(row[4], datatype=XSD.string) ])
    g.add([res, sso.hasName, Literal(row[5], datatype=XSD.string) ])

    if(row[6] != 0 and row[7] != 0):
        g.add([res, sso.hasLatitude, Literal(row[6], datatype=XSD.decimal) ])
        g.add([res, sso.hasLongitude, Literal(row[7], datatype=XSD.decimal) ])
    g.add([res, sso.hasVatNumber, Literal(row[8], datatype=XSD.integer) ])


g = Graph()
sso = Namespace("http://www.sanitasicilia.it/ontology/")
g.bind("sso", sso)
ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)

farmacie_df = pd.read_csv("../datasets/completed/farmacie-out.csv")


farmacie_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination='../datasets/rdf/farmacie.ttl', 
            format='turtle')

<Graph identifier=N5ecdc6286107452fbacceeacc2f13b42 (<class 'rdflib.graph.Graph'>)>

In [ ]:
#Parafarmacie
def addTriples(row):
    #sostituire con codice farmacia/parafarmacia piuttosto che nome 
    res = URIRef(urify(base_uri, row[5]))
    g.add([res, URIRef(RDF.type), URIRef(sso.Parafarmacia)])
    g.add([res, sso.isIn, URIRef(urify(base_uri, row[2].title()))])

    if(row[3] != 0):
        g.add([res, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
        
    g.add([res, sso.hasAddress, Literal(row[4], datatype=XSD.string) ])
    g.add([res, sso.hasName, Literal(row[5], datatype=XSD.string) ])

    if(row[6] != 0 and row[7] != 0):
        g.add([res, sso.hasLatitude, Literal(row[6], datatype=XSD.decimal) ])
        g.add([res, sso.hasLongitude, Literal(row[7], datatype=XSD.decimal) ])
    g.add([res, sso.hasVatNumber, Literal(row[8], datatype=XSD.integer) ])

parafarmacie_df = pd.read_csv("../datasets/completed/parafarmacie.csv", sep=';')
parafarmacie_df["LATITUDINE"] = parafarmacie_df["LATITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
parafarmacie_df["LONGITUDINE"] = parafarmacie_df["LONGITUDINE"].apply(lambda x : float(x.replace(',', '.')) if x != '-' else 0.0)
parafarmacie_df["PARTITAIVA"] = parafarmacie_df["PARTITAIVA"].apply(lambda x : x if x != '-' else 0)


g = Graph() #attualmente, per debug rdf separati
sso = Namespace("http://www.sanitasicilia.it/ontology/")
g.bind("sso", sso)
ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)

parafarmacie_df.apply(lambda row : addTriples(row), axis=1)
g.serialize(destination='../datasets/rdf/parafarmacie.ttl', 
            format='turtle')
            

In [24]:
#Strutture sanitarie private
def addTriples(row):
    #sostituire con codice struttura
    res = URIRef(urify(base_uri, row[7]))
    g.add([res, URIRef(RDF.type), URIRef(sso.StrutturaPrivata)])

    if(row[0] != 0):
        g.add([res, sso.hasCap, Literal(row[0], datatype=XSD.integer) ])
        
    if(row[1] != 0 and row[2] != 0):
        g.add([res, sso.hasLatitude, Literal(row[1], datatype=XSD.decimal) ])
        g.add([res, sso.hasLongitude, Literal(row[2], datatype=XSD.decimal) ])
    
    g.add([res, sso.hasReferenceAsp, Literal(row[3], datatype=XSD.string)])

    if(row[5] != ''):
        g.add([res, sso.isIn, URIRef(urify(base_uri, row[5].title()))])

    if(row[6] != 0):
        g.add([res, sso.hasAddress, Literal(row[6], datatype=XSD.string) ])
        
    g.add([res, sso.hasName, Literal(row[7], datatype=XSD.string) ])
    g.add([res, sso.businessType, Literal(row[8], datatype=XSD.string)])
    g.add([res, sso.hasSiteType, Literal(row[9], datatype=XSD.string) ])

g = Graph()
sso = Namespace("http://www.sanitasicilia.it/ontology/")
g.bind("sso", sso)
ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)

private_df = pd.read_csv("../datasets/completed/private.csv")
private_df["Citta\'"] = private_df["Citta\'"].astype(str)
private_df.apply(lambda row : addTriples(row), axis=1)
g.serialize(destination='../datasets/rdf/private.ttl', 
            format='turtle')

<Graph identifier=Ndede4723e28640238ec3c81913218d1f (<class 'rdflib.graph.Graph'>)>

In [28]:
import math

In [34]:
#Strutture sanitarie pubbliche
def addTriples(row):
    #sostituire con codice struttura
    res = URIRef(urify(base_uri, row[5]))
    g.add([res, URIRef(RDF.type), URIRef(sso.StrutturaPubblica)])

    if(row[0] != ''):
        g.add([res, sso.isIn, URIRef(urify(base_uri, row[0].title()))])

    if(row[1] != 0 and row[2] != 0):
        g.add([res, sso.hasLatitude, Literal(row[1], datatype=XSD.decimal) ])
        g.add([res, sso.hasLongitude, Literal(row[2], datatype=XSD.decimal) ])
   
    g.add([res, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
    g.add([res, sso.hasAddress, Literal(row[4], datatype=XSD.string)])
    g.add([res, sso.hasName, Literal(row[5], datatype=XSD.string) ])
    g.add([res, sso.businessType, Literal(row[7], datatype=XSD.string)])
    g.add([res, sso.hasSiteType, Literal(row[8], datatype=XSD.string)])

    if row[9] != '':
        g.add([res, sso.isRecoveryStructure, Literal(True, datatype=XSD.boolean)])
        g.add([res, sso.hasRecoveryStructureName, Literal(row[10], datatype=XSD.string)])
    else:
        g.add([res, sso.isRecoveryStructure, Literal(False, datatype=XSD.boolean)])

    g.add([res, sso.workingMonths, Literal(row[10], datatype=XSD.integer)])
    g.add([res, sso.openingDays, Literal(row[12], datatype=XSD.integer)])
    g.add([res, sso.weeklyHours, Literal(row[13], datatype=XSD.integer)])
    g.add([res, sso.hasVatNumber, Literal(row[14], datatype=XSD.integer)])
    g.add([res, sso.hasEmail, Literal(row[15], datatype=XSD.string)])

    if row[16] != '':
        g.add([res, sso.hasWebsite, Literal(row[16], datatype=XSD.string)])
   
    if not math.isnan(row[17]):
        g.add([res, sso.hasPhonePrefix, Literal(row[17], datatype=XSD.integer)])
   
    if not math.isnan(row[18]):
        g.add([res, sso.hasPhoneNumber, Literal(row[18], datatype=XSD.integer)])
    
    if not math.isnan(row[19]):
        g.add([res, sso.hasFaxPrefix, Literal(row[19], datatype=XSD.integer)])
    
    if not math.isnan(row[20]):
        g.add([res, sso.hasFaxNumber, Literal(row[20], datatype=XSD.integer)])

g = Graph()
sso = Namespace("http://www.sanitasicilia.it/ontology/")
g.bind("sso", sso)
ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)

pubbliche_df = pd.read_csv("../datasets/completed/pubbliche.csv")

pubbliche_df["Comune"] = pubbliche_df["Comune"].astype(str)
pubbliche_df["Denominazione struttura di ricovero"] = pubbliche_df["Denominazione struttura di ricovero"].astype(str)
pubbliche_df["Sito web"] = pubbliche_df["Sito web"].astype(str)

pubbliche_df.apply(lambda row : addTriples(row), axis=1)
g.serialize(destination='../datasets/rdf/pubbliche.ttl', format='turtle')

<Graph identifier=N2f7608768f2346a68ff27767de3ab2aa (<class 'rdflib.graph.Graph'>)>